In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val

from scipy.optimize import curve_fit
from scipy.stats import norm
import scipy.stats as stats


<h3> funzioni

In [2]:
def gaussian(x, amp, mu, sigma, fondo):   # funzione gaussiana
    return amp * norm.pdf(x,loc = mu,scale = sigma) + fondo

def read(file, s, m, skip_header=True):    # funzione di lettura
    if skip_header:
        s += 1  # Skip the header row
    return np.loadtxt(file, unpack=True, skiprows=s, max_rows=m, delimiter=',')

def fittiamo(x_data, y_data, p0 = [1, 1, 1, 1]):    # funzione fit e ottimizzazione
    popt, pcov = curve_fit(gaussian, x_data, y_data, p0)
    # amp_fit, mu_fit, sigma_fit, offset_fit = popt      giusto per capire cosa spara fuori popt
    return popt, pcov    

def plottiamo_plotly(data):
    
    fig = go.Figure()
    X, Y = data
    popt, pcov = fittiamo(*data)
    fwhm = FWHM(popt)
    
    fig.add_trace(go.Scatter(x = X, y = Y, mode = 'lines', line_color = 'plum', 
                             showlegend=False, hoverinfo='skip'))

    fig.add_trace(go.Scatter(x = X, y = gaussian(X, *popt), mode='lines', showlegend=False,
        marker = dict(color = 'purple')))

    fig.update_layout(
            yaxis_title = "Gray Value",
            xaxis_title = "Distance (μm)",
            plot_bgcolor = 'white',
            font = dict(
                size = 18,
                color = "Black"
            )
        )

    fig.update_xaxes(
        mirror = True,
        ticks = 'outside',
        showline = True,
        linecolor = 'black',
        gridcolor = 'lightgrey'
    )
    fig.update_yaxes(
        mirror = True,
        ticks = 'outside',
        showline = True,
        linecolor = 'black',
        gridcolor = 'lightgrey'
    )

    # Aggiungi annotazione per il valore della FWHM in alto a destra
    fig.add_annotation(
        x=max(X),  # Posizione x al massimo valore di X
        y=max(Y),  # Posizione y al massimo valore di Y
        text=f"FWHM = {fwhm:.2f}",  # Testo da mostrare
        showarrow=False,
        font=dict(
            size=14,
            color="black"
        ),
        align="right",
        xanchor="right",
        yanchor="top"
    )

    fig.show()
    return popt, pcov

def FWHM(popt):
    fwhm = 2 * np.sqrt(2 * np.log(2)) * popt[2] * 1000
    return fwhm

<h3> Risoluzione olio

In [3]:
def risoluz_olio():
    n_olio = 1.47
    NA = n_olio * np.sin(np.pi / 6)
    l_verde = 550
    return ((1.22 * l_verde) / (2 * NA)) * 0.6

risoluzione = []
risol_giusta = risoluz_olio()
print(f"risoluzione attesa = {risol_giusta:.2f}")

risoluzione attesa = 273.88


# Casi: FWHM gaussiane

In [4]:
skiprows = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
maxrows = [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]

In [5]:
def plot_and_resolution(i):
    data = np.array(read(f'data_vera/r{i}.csv', s = skiprows[i - 1], m = maxrows[i - 1]))
    popt, pcov = plottiamo_plotly(data)  # scegliere plotly o pyplot
    fwhm = FWHM(popt)
    risoluzione.append(fwhm)
    print(f"FWHM = {fwhm:.2f}")

In [6]:
for i in range(1, 11):
    plot_and_resolution(i)
    print(i)

FWHM = 223.52
1


FWHM = 252.13
2


FWHM = 363.27
3


FWHM = 228.01
4


FWHM = 293.54
5


FWHM = 204.00
6


FWHM = 184.96
7


FWHM = 254.66
8


FWHM = 202.90
9


FWHM = 217.18
10
